In [1]:
#movie recommender based on imdb weighted rating formula and genre

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import ast 
from scipy import stats
from ast import literal_eval

In [2]:
md = pd.read_csv('movies_metadataa.csv')
md.head()

C:\Users\MANI\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                   NaN   8844  tt0113497                en   
2                                   NaN  15602  tt0113228                en   
3                                   NaN  31357  tt0114885                en   
4                                   NaN  11862  tt0113041                en   

                original_title  \
0                    Toy Story   
1                      Jumanji   
2             Grumpier Old Men   
3            Waiting to Exhale   
4  Father of the Bride Part II   

                                            overview  ... release_date  \
0  Led by Woody, Andy's toys live happily in his ...  ...   1995-10-30   
1  When siblings Judy and Peter discover an encha...  ...   1995-12-15   
2  A family wedding reignites the ancient feud be...  ...   1995-12-22   
3  Cheated on, mistreated and stepped on, the wom...  ...   1995-12-22   
4  Just when George Banks has recovered from his ...  ...   1995-02-10   

       revenue runtime                                   spoken_languages  \
0  373554033.0    81.0           [{'iso_639_1': 'en', 'name': 'English'}]   
1  262797249.0   104.0  [{'iso_639_1': 'en', 'name': 'English'}, {'iso...   
2          0.0   101.0           [{'iso_639_1': 'en', 'name': 'English'}]   
3   81452156.0   127.0           [{'iso_639_1': 'en', 'name': 'English'}]   
4   76578911.0   106.0           [{'iso_639_1': 'en', 'name': 'English'}]   

     status                                            tagline  \
0  Released                                                NaN   
1  Released          Roll the dice and unleash the excitement!   
2  Released  Still Yelling. Still Fighting. Still Ready for...   
3  Released  Friends are the people who let you be yourself...   
4  Released  Just When His World Is Back To Normal... He's ...   

                         title  video vote_average vote_count  
0                    Toy Story  False          7.7     5415.0  
1                      Jumanji  False          6.9     2413.0  
2             Grumpier Old Men  False          6.5       92.0  
3            Waiting to Exhale  False          6.1       34.0  
4  Father of the Bride Part II  False          5.7      173.0  

[5 rows x 24 columns]

In [3]:
md.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [4]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i[
    'name'] for i in x] if isinstance(x, list) else [])
print(md['genres'])

0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2                   [Romance, Comedy]
3            [Comedy, Drama, Romance]
4                            [Comedy]
                     ...             
45461                 [Drama, Family]
45462                         [Drama]
45463       [Action, Drama, Thriller]
45464                              []
45465                              []
Name: genres, Length: 45466, dtype: object


In [5]:
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(
    lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [6]:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
Cum = vote_averages.mean()
Cum

5.244896612406511

In [7]:
minimum_votes = vote_counts.quantile(0.95)
minimum_votes

434.0

In [8]:
new_md = md[(md['vote_count'] >= minimum_votes) & 
               (md['vote_count'].notnull()) & 
               (md['vote_average'].notnull())][['title', 
                                                'year', 
                                                'vote_count', 
                                                'vote_average', 
                                                'popularity', 
                                                'genres']]

new_md['vote_count'] = new_md['vote_count'].astype('int')
new_md['vote_average'] = new_md['vote_average'].astype('int')
new_md.shape

(2274, 6)

In [9]:
new_md.head()

title  year  vote_count  vote_average popularity  \
0   Toy Story  1995        5415             7  21.946943   
1     Jumanji  1995        2413             6  17.015539   
5        Heat  1995        1886             7  17.924927   
9   GoldenEye  1995        1194             6  14.686036   
15     Casino  1995        1343             7  10.137389   

                              genres  
0        [Animation, Comedy, Family]  
1       [Adventure, Fantasy, Family]  
5   [Action, Crime, Drama, Thriller]  
9      [Adventure, Action, Thriller]  
15                    [Drama, Crime]

In [10]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+minimum_votes) * R) + (minimum_votes/(minimum_votes+v) * Cum)

In [11]:
new_md['WR'] = new_md.apply(weighted_rating, axis=1)

In [12]:
new_md = new_md.sort_values('WR', ascending = False)

In [13]:
new_md.head()

title  year  vote_count  \
15480                                          Inception  2010       14075   
12481                                    The Dark Knight  2008       12269   
22879                                       Interstellar  2014       11187   
2843                                          Fight Club  1999        9678   
4863   The Lord of the Rings: The Fellowship of the Ring  2001        8892   

       vote_average  popularity  \
15480             8   29.108149   
12481             8  123.167259   
22879             8   32.213481   
2843              8   63.869599   
4863              8   32.070725   

                                                  genres        WR  
15480  [Action, Thriller, Science Fiction, Mystery, A...  7.917588  
12481                   [Drama, Action, Crime, Thriller]  7.905871  
22879                [Adventure, Drama, Science Fiction]  7.897107  
2843                                             [Drama]  7.881753  
4863                        [Adventure, Fantasy, Action]  7.871787

In [14]:
new_md.head(15)

title  year  vote_count  \
15480                                          Inception  2010       14075   
12481                                    The Dark Knight  2008       12269   
22879                                       Interstellar  2014       11187   
2843                                          Fight Club  1999        9678   
4863   The Lord of the Rings: The Fellowship of the Ring  2001        8892   
292                                         Pulp Fiction  1994        8670   
314                             The Shawshank Redemption  1994        8358   
7000       The Lord of the Rings: The Return of the King  2003        8226   
351                                         Forrest Gump  1994        8147   
5814               The Lord of the Rings: The Two Towers  2002        7641   
256                                            Star Wars  1977        6778   
1225                                  Back to the Future  1985        6239   
834                                        The Godfather  1972        6024   
1154                             The Empire Strikes Back  1980        5998   
46                                                 Se7en  1995        5915   

       vote_average  popularity  \
15480             8   29.108149   
12481             8  123.167259   
22879             8   32.213481   
2843              8   63.869599   
4863              8   32.070725   
292               8  140.950236   
314               8   51.645403   
7000              8   29.324358   
351               8   48.307194   
5814              8   29.423537   
256               8   42.149697   
1225              8   25.778509   
834               8   41.109264   
1154              8   19.470959   
46                8    18.45743   

                                                  genres        WR  
15480  [Action, Thriller, Science Fiction, Mystery, A...  7.917588  
12481                   [Drama, Action, Crime, Thriller]  7.905871  
22879                [Adventure, Drama, Science Fiction]  7.897107  
2843                                             [Drama]  7.881753  
4863                        [Adventure, Fantasy, Action]  7.871787  
292                                    [Thriller, Crime]  7.868660  
314                                       [Drama, Crime]  7.864000  
7000                        [Adventure, Fantasy, Action]  7.861927  
351                             [Comedy, Drama, Romance]  7.860656  
5814                        [Adventure, Fantasy, Action]  7.851924  
256                 [Adventure, Action, Science Fiction]  7.834205  
1225        [Adventure, Comedy, Science Fiction, Family]  7.820813  
834                                       [Drama, Crime]  7.814847  
1154                [Adventure, Action, Science Fiction]  7.814099  
46                            [Crime, Mystery, Thriller]  7.811669

In [16]:
g = md.apply(lambda x:pd.Series(x['genres']),axis = 1).stack().reset_index(level = 1,drop = True)
g.name='genre'
gen_md = md.drop('genres',axis = 1).join(g)
gen_md.head()

<ipython-input-16-7d8b7c244552>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  g = md.apply(lambda x:pd.Series(x['genres']),axis = 1).stack().reset_index(level = 1,drop = True)


adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
1  False                                                NaN  65000000   

                               homepage    id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story   862  tt0114709                en   
0  http://toystory.disney.com/toy-story   862  tt0114709                en   
0  http://toystory.disney.com/toy-story   862  tt0114709                en   
1                                   NaN  8844  tt0113497                en   
1                                   NaN  8844  tt0113497                en   

  original_title                                           overview  \
0      Toy Story  Led by Woody, Andy's toys live happily in his ...   
0      Toy Story  Led by Woody, Andy's toys live happily in his ...   
0      Toy Story  Led by Woody, Andy's toys live happily in his ...   
1        Jumanji  When siblings Judy and Peter discover an encha...   
1        Jumanji  When siblings Judy and Peter discover an encha...   

  popularity  ... runtime                                   spoken_languages  \
0  21.946943  ...    81.0           [{'iso_639_1': 'en', 'name': 'English'}]   
0  21.946943  ...    81.0           [{'iso_639_1': 'en', 'name': 'English'}]   
0  21.946943  ...    81.0           [{'iso_639_1': 'en', 'name': 'English'}]   
1  17.015539  ...   104.0  [{'iso_639_1': 'en', 'name': 'English'}, {'iso...   
1  17.015539  ...   104.0  [{'iso_639_1': 'en', 'name': 'English'}, {'iso...   

     status                                    tagline      title  video  \
0  Released                                        NaN  Toy Story  False   
0  Released                                        NaN  Toy Story  False   
0  Released                                        NaN  Toy Story  False   
1  Released  Roll the dice and unleash the excitement!    Jumanji  False   
1  Released  Roll the dice and unleash the excitement!    Jumanji  False   

  vote_average vote_count  year      genre  
0          7.7     5415.0  1995  Animation  
0          7.7     5415.0  1995     Comedy  
0          7.7     5415.0  1995     Family  
1          6.9     2413.0  1995  Adventure  
1          6.9     2413.0  1995    Fantasy  

[5 rows x 25 columns]

In [20]:
def final_1(genre, percentile=0.85):
    df=gen_md[gen_md['genre']==genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    Cum = vote_averages.mean()
    minimum_votes = vote_counts.quantile(percentile)
    final1_md = df[(df['vote_count'] >= minimum_votes) & (df['vote_count'].notnull()) & 
                   (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    final1_md['vote_count'] = final1_md['vote_count'].astype('int')
    final1_md['vote_average'] = final1_md['vote_average'].astype('int')
    
    final1_md['WR'] = final1_md.apply(lambda x: 
                        (x['vote_count']/(x['vote_count']+minimum_votes) * x['vote_average']) + (minimum_votes/(minimum_votes+x['vote_count']) * Cum),
                        axis=1)
    final1_md = final1_md.sort_values('WR', ascending=False).head(20)
    
    return final1_md

In [21]:
final_1('Comedy').head(10)

title  year  vote_count  \
10309                        Dilwale Dulhania Le Jayenge  1995         661   
351                                         Forrest Gump  1994        8147   
1225                                  Back to the Future  1985        6239   
18465                                   The Intouchables  2011        5410   
22841                           The Grand Budapest Hotel  2014        4644   
2211                                   Life Is Beautiful  1997        3643   
732    Dr. Strangelove or: How I Learned to Stop Worr...  1964        1472   
3342                                        Modern Times  1936         881   
883                                     Some Like It Hot  1959         835   
1236                                  The Great Dictator  1940         756   

       vote_average popularity        WR  
10309             9  34.457024  8.463024  
351               8  48.307194  7.963363  
1225              8  25.778509  7.952358  
18465             8  16.086919  7.945207  
22841             8  14.442048  7.936384  
2211              8   39.39497  7.919430  
732               8    9.80398  7.809073  
3342              8   8.159556  7.695554  
883               8  11.845107  7.680781  
1236              8   9.241748  7.651762